# RSSIデータを用いた場所分類

`data/` ディレクトリにあるRSSIデータ（ファイル名に場所が含まれる）を読み込み、1分間のRSSIデータパターンから場所を分類するモデルを構築します。

## 1. ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# import joblib # model_utils で使用
import japanize_matplotlib

# --- ライブラリからのインポート ---
from blelocation.feature_engineering import extract_inference_features as extract_features_lib # 名前が衝突するため別名でインポート
from blelocation.model_utils import save_model_bundle


## 2. データの読み込みと前処理

`data/` ディレクトリ内の全CSVファイルを読み込み、ファイル名からクラスラベル（場所）を抽出します。
その後、絶対時刻をdatetime型に変換します。

In [ ]:
data_dir = Path('data')
all_files = list(data_dir.glob('*.csv'))

all_dfs = []
for f in all_files:
    # ファイル名からクラスラベル（場所）を抽出 (例: 202504261747_クローゼット.csv -> クローゼット)
    match = re.search(r'_(.+)\.csv', f.name)
    if match:
        location = match.group(1)
        try:
            df = pd.read_csv(f)
            # datetime列が存在し、内容があるか確認
            if 'datetime' in df.columns and not df['datetime'].isnull().all():
                df['location'] = location
                # datetime列をdatetime型に変換 (エラーは無視してNaTにする)
                df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
                # time_s, rssi_dbm列が存在するか確認
                if 'time_s' in df.columns and 'rssi_dbm' in df.columns:
                     # NaTになった行を削除
                    df.dropna(subset=['datetime'], inplace=True)
                    if not df.empty:
                       all_dfs.append(df[['datetime', 'rssi_dbm', 'location']])
                    else:
                        print(f"[WARN] No valid datetime entries in {f.name} after conversion.")
                else:
                    print(f"[WARN] 'time_s' or 'rssi_dbm' column missing in {f.name}. Skipping.")
            else:
                 print(f"[WARN] 'datetime' column missing or empty in {f.name}. Skipping.")
        except Exception as e:
            print(f"[ERROR] Failed to read or process {f.name}: {e}")
    else:
        print(f"[WARN] Could not extract location from filename: {f.name}. Skipping.")

if not all_dfs:
    print("[ERROR] No valid data found in CSV files. Exiting.")
    # ここで処理を中断するか、空のDataFrameを作成するかを決定
    # exit() # スクリプトの場合
    data = pd.DataFrame(columns=['datetime', 'rssi_dbm', 'location'])
else:
    data = pd.concat(all_dfs, ignore_index=True)
    # datetimeでソート
    data = data.sort_values(by='datetime').reset_index(drop=True)

print(f"Total records loaded: {len(data)}")
print("Unique locations:", data['location'].unique())
data.head()

### 2.1 データのサンプリング

データを1分ごとのウィンドウに分割し、各ウィンドウに **MIN_POINTS_PER_WINDOW** 以上のデータポイントが含まれるものを1サンプルとします。
各サンプルから特徴量（RSSIの平均、標準偏差、最小値、最大値など）を抽出します。

In [ ]:
# サンプリングウィンドウ内の最小データポイント数
MIN_POINTS_PER_WINDOW = 5

# datetimeをインデックスに設定
if not data.empty and 'datetime' in data.columns:
    data.set_index('datetime', inplace=True)
else:
    print("[WARN] Data is empty or 'datetime' column missing before resampling.")

# 1分ごとにリサンプリングし、各ウィンドウで特徴量を計算
# Grouperを使って1分間のウィンドウを作成
# def extract_features(window): ... Function definition removed ...

if not data.empty:
    # location ごとにグループ化してからリサンプリングしないと、異なる場所のデータが混ざる可能性がある
    sampled_features_list = []
    for location, group in data.groupby('location'):
        # 1分間のウィンドウでリサンプリングし、各ウィンドウに関数を適用
        # 'T' は分単位のオフセットエイリアス
        resampler = group.resample('1min')
        # ライブラリ関数を使用 (lambdaで追加引数を渡す)
        # features = resampler.apply(lambda window: extract_features_lib(window, rssi_col='rssi_dbm', feature_names=None, min_points_required=MIN_POINTS_PER_WINDOW))
        features = [extract_features_lib(window, rssi_col='rssi_dbm', feature_names=None, min_points_required=MIN_POINTS_PER_WINDOW) for _,window in resampler]
        features = pd.concat(features, ignore_index=True).assign(location=location)
        # 欠損値（データが閾値未満だったウィンドウ）を削除
        features.dropna(inplace=True)
        if not features.empty:
           sampled_features_list.append(features)

    if sampled_features_list:
        sampled_data = pd.concat(sampled_features_list)
        # stdがNaNになる場合（ウィンドウ内のデータが1つの場合など）を0で埋める
        sampled_data['rssi_std'].fillna(0, inplace=True)
        print(f"Total samples after resampling (>={MIN_POINTS_PER_WINDOW} points/min): {len(sampled_data)}")
        print("Sample counts per location:")
        print(sampled_data['location'].value_counts())
        sampled_data.head()
    else:
       print(f"[WARN] No samples met the criteria (>={MIN_POINTS_PER_WINDOW} points/min).")
       sampled_data = pd.DataFrame() # 空のDataFrame
else:
    print("[WARN] Data is empty, skipping feature extraction.")
    sampled_data = pd.DataFrame() # 空のDataFrame

## 3. EDA (探索的データ分析)

In [ ]:
# sampled_dataが空でないかチェック
if not sampled_data.empty:
    # 特徴量の分布を確認 (数値型の特徴量のみ)
    numeric_features = sampled_data.select_dtypes(include=np.number).columns
    sampled_data[numeric_features].hist(bins=15, figsize=(15, 10), layout=(-1, 3))
    plt.suptitle('Distribution of Features')
    plt.tight_layout(rect=(0, 0.03, 1, 0.95)) # Adjust layout to prevent title overlap
    plt.show()

    # クラスごとの特徴量の比較 (箱ひげ図)
    plt.figure(figsize=(18, 12))
    for i, feature in enumerate(numeric_features):
        if feature != 'count': # countはサンプル数なので除外することが多い
           plt.subplot(2, 3, i + 1) # レイアウト調整
           sns.boxplot(x='location', y=feature, data=sampled_data)
           plt.title(f'{feature} by Location')
           plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # 特徴量間の相関
    plt.figure(figsize=(10, 8))
    correlation_matrix = sampled_data[numeric_features].corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Feature Correlation Matrix')
    plt.show()
else:
    print("[INFO] sampled_data is empty. Skipping EDA plots.")

## 4. モデル構築と評価

In [ ]:
# sampled_dataが空でないかチェック
if not sampled_data.empty:
    # 特徴量 (X) とターゲット (y) に分割
    X = sampled_data.drop(['location'], axis=1)
    y = sampled_data['location']

    # データセットをトレーニングセットとテストセットに分割
    # クラスの比率を保つために stratify=y を指定
    # 十分なサンプルがないクラスがあるとエラーになる可能性あり
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
        print(f"Train set size: {X_train.shape[0]}")
        print(f"Test set size: {X_test.shape[0]}")

        # 特徴量のスケーリング
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # ランダムフォレストモデルのトレーニング
        model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        model.fit(X_train_scaled, y_train)

        # テストデータで予測
        y_pred = model.predict(X_test_scaled)

        # モデルの評価
        print("\nModel Evaluation:")
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))

        # 混同行列の計算
        cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
        cm_abs = cm # 絶対数を保持

        # Precision (列方向で正規化) を計算 (ゼロ除算を避ける)
        cm_sum_col = cm.sum(axis=0, keepdims=True)
        # ゼロ除算が発生する列（合計が0）は NaN になるようにする
        with np.errstate(divide='ignore', invalid='ignore'):
            cm_precision = cm.astype('float') / cm_sum_col
            cm_precision[np.isnan(cm_precision)] = 0 # NaN は 0 に置換

        # Precisionで色付けし、絶対数値を表示
        plt.figure(figsize=(10, 8)) # サイズ調整
        sns.heatmap(cm_precision, annot=cm_abs, fmt='d', cmap='Blues',
                    xticklabels=model.classes_.tolist(), yticklabels=model.classes_.tolist(),
                    cbar_kws={'label': 'Precision (Column Normalized)'}) # カラーバーにラベル追加
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix (Color by Precision, Number is Count)')
        plt.show()

        # 特徴量の重要度
        feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
        plt.figure(figsize=(10, 6))
        sns.barplot(x=feature_importances, y=feature_importances.index)
        plt.title('Feature Importances')
        plt.xlabel('Importance Score')
        plt.ylabel('Features')
        plt.show()

        # --- モデル、スケーラー、メタデータをまとめて保存 ---
        bundle_filename = 'location_classification_bundle.joblib'
        # MIN_POINTS_PER_WINDOW はこのセルより前のセル（サンプリング部）で定義されている想定
        bundle = {
            'model': model,
            'scaler': scaler,
            'classes': model.classes_,
            'features': X.columns.tolist(), # Indexをリストに変換
            'min_points_per_window': MIN_POINTS_PER_WINDOW
        }
        # ライブラリ関数で保存
        save_model_bundle(bundle, bundle_filename)

    except ValueError as e:
        print(f"\n[ERROR] Could not split data or train model: {e}")
        print("This might be due to insufficient samples in one or more classes for stratification.")
    except NameError as e:
        # MIN_POINTS_PER_WINDOW が定義されていない場合のエラーハンドリング
        if 'MIN_POINTS_PER_WINDOW' in str(e):
            print(f"\n[ERROR] {e}. Please make sure the cell defining MIN_POINTS_PER_WINDOW (in section 2.1) is executed before this cell.")
        else:
            print(f"\n[ERROR] An unexpected NameError occurred: {e}")

else:
    print("[INFO] sampled_data is empty. Skipping model training and evaluation.")
